In [1]:
# from PyPDF2 import PdfReader
# import src.ConvertPdftoTxt as cpt
from tqdm import tqdm
import os
import re
import json

In [2]:
# import pandas as pd
# pd.DataFrame(os.listdir(output_folder))

In [4]:
pdf_folder_path = '/home/minds/army_smart/data/jisi/'
output_folder = '/home/minds/army_smart/data/jisi/'

## pdf -> txt 변환
# cpt.save_all_pdfs_as_texts(pdf_folder_path, output_folder)

In [21]:
## txt file open
# txt_file_name = os.listdir(output_folder)[0] ## file_name
txt_file_name = '17-3001.txt'

txt_list = []
with open(os.path.join(output_folder, txt_file_name),'r') as f:
    line = f.readlines()
    txt_list += line
f.close()

txt_list

['       \n',
 '         \n',
 '제 목 :        보안상 ・벌점 부과 업무처리 지시\n',
 '육   군   본   부발령일자 : 2017. 5. 1 \n',
 '유효기간 : 2019. 5. 1 (2년) 분류 : 육본지시 제17 - 3001호\n',
 '발신 : 참모총장\n',
 '수신 : 수신처 참조\n',
 '보안상 ㆍ벌점 부과 업무처리 지시\n',
 '(육군 지시 제17 - 3001호, ’17. 5. 1)\n',
 '1. 개    요\n',
 '  「육규 200 군사보안규정 」(이하 “육규”라고 한다) 제194조의2 1항에 따\n',
 '른 보안벌점 및「보안위반자 인사관리 강화 지침」에서 “보안 상⋅벌\n',
 '점’ 인사관리 (진급심사 ) 반영”에 따라 서면경고 , 기타(불문, 구두경고 )에 \n',
 '대한 이의제기 절차와 보안벌점 ㆍ상점 처리절차를  지시하는  사항임 .\n',
 '2. 개정 배경\n',
 '   보안벌점 부과에 대한 이의제기 절차 및 연대급이하의 보안상점 건의 시 \n',
 '장관급 지휘관의 상점부여 내용을 추가하였음.\n',
 '3. 기본지시\n',
 '   가. 서면경고 , 기타(불문, 구두경고 )에 따라 보안벌점을 부과하는 절차 \n',
 '및 보안상점 관리절차는 다음과 같다. \n',
 ' ① 상ㆍ벌점\n',
 '    부 여\n',
 '  (각급 부대장 )⇨ ② 상ㆍ벌점\n',
 '   개인 통보\n',
 ' (이의제기 접수)⇨ ③ 보안상 ㆍ벌점\n',
 '   심의위 개최\n',
 ' (이의제기 접수)⇨ ④ 심의결과\n',
 ' 보고 / 개인통보\n',
 ' (장성급 부대장 )⇨ ⑤ 명령발령\xa0\xa0/\n',
 '   기변처리⇨ ⑥ 상ㆍ벌점\n',
 '    입력\n',
 '보안위반\n',
 '확인서\n',
 '(보안감사관 작성 \n',
 '/ 개인이 확인)보안부서에서\n',
 '개인에게 공문 또는\n',
 '서면 통보\n',
 '(이의제기서 접수 :\n',
 '직접수발 / 우편)이의제

In [22]:
## 로마자가 있는 경우, 장으로 바꾸고, 아날로그 숫자만 있는 경우 조로 바꾸기
def roman_to_int(s):
    roman_numerals = {'Ⅰ': 1, 'Ⅱ': 2, 'Ⅲ': 3, 'Ⅳ': 4, 'Ⅴ': 5, 'Ⅵ': 6, 'Ⅶ': 7, 'Ⅷ': 8, 'Ⅸ': 9, 'Ⅹ': 10}
    return roman_numerals.get(s, 0)

transformed_list = []
for txt in txt_list:
    # 정규식 패턴으로 로마 숫자를 포함하는 행 찾기
    roman_pattern = r'^\s*(Ⅰ|Ⅱ|Ⅲ|Ⅳ|Ⅴ|Ⅵ|Ⅶ|Ⅷ|Ⅸ|Ⅹ)\. .+?$\n'
    match = re.match(roman_pattern, txt)
    if match:
        roman_numeral = match.group(1)
        arabic_numeral = roman_to_int(roman_numeral)
        # 로마 숫자를 "제 {숫자} 장" 형태로 교체
        # 아라빅 숫자를 "제 {숫자} 조" 형태로 교체
        txt = re.sub(r'^\s*Ⅰ|Ⅱ|Ⅲ|Ⅳ|Ⅴ|Ⅵ|Ⅶ|Ⅷ|Ⅸ|Ⅹ', f'제 {arabic_numeral}장', txt)
        txt = re.sub(r'^\s*(\d+)\.(.+)\n', r'제 \1조\2', txt)
        transformed_list.append(txt)
    else:
        txt = re.sub(r'^(\s*)(\d+)\.(.+)\n', r'\1제 \2조\3', txt)
        transformed_list.append(txt)

In [24]:
transformed_list[104]

'  나. 본 지시는 2017. 5. 1.일부터 시행하되 , 2017. 1. 1.일부터 받는 보안\n'

In [9]:
## 이 문서는 목차가 없음. 그리고 조 단위가 없으므로 1. ~ 인걸로 찾아야 함
## 조항 내용 / 별표와 별지 따로 리스트 생성
if len([i for i, txt in enumerate(txt_list) if re.search('\s*1\.', txt)]) >= 2:
    jeoul_idx = [i for i, txt in enumerate(txt_list) if re.search('\s*1\.', txt)]
    start_raw = jeoul_idx[0]

## 별표 및 별지 별도 추출
## 별표 및 별지가 없을 경우, 해당 텍스트 자체가 모두 raw_txt
if len([i for i, txt in enumerate(txt_list) if re.search('(별표\s*및\s*별지)|(별표\s*목차)|(별지\s*목차)|(별표\s*및)|(별표\s*\/\s*서식)', txt)]) >= 2:
    start_pyo = [i for i, txt in enumerate(txt_list) if re.search('(별표\s*및\s*별지)|(별표\s*목차)|(별지\s*목차)|(별표\s*및)|(별표\s*\/\s*서식)', txt)]
    start_pyo = max(start_pyo)
    pyo_contents = [i for i, txt in enumerate(txt_list) if re.search('\〔별지|\[별지|\【별지|\〔별표|\[별표|\【별표|\〔별표\s*제?1호?\〕|\[별표\s*제?1호?\]|\【별표\s*제?1호?\】', txt)]

    raw_txt = txt_list[start_raw:start_pyo]
    pyo_txt = txt_list[max(pyo_contents):]

elif len([i for i, txt in enumerate(txt_list) if re.search('(별표\s*및\s*별지)|(별표\s*목차)|(별지\s*목차)|(별표\s*및)|(별표\s*\/\s*서식)', txt)]) < 1:
    start_kook = [i for i, txt in enumerate(txt_list) if re.search('(육\s*군\s*참\s*모\s*총\s*장)|(국\s*방\s*부\s*장\s*관)', txt)]
    start_kook = max(start_kook)
    pyo_contents = [i for i, txt in enumerate(txt_list) if re.search('\〔별지|\[별지|\【별지|\〔별표|\[별표|\【별표|\〔별표\s*제?1호?\〕|\[별표\s*제?1호?\]|\【별표\s*제?1호?\】', txt)]

    raw_txt = txt_list[start_raw:start_kook]
    pyo_txt = txt_list[max(pyo_contents):]
else:
    raw_txt = txt_list[start_raw:]
    pyo_txt = []

'1. 개    요\n'

In [28]:
raw_txt

['1. 개    요\n',
 '  「육규 200 군사보안규정 」(이하 “육규”라고 한다) 제194조의2 1항에 따\n',
 '른 보안벌점 및「보안위반자 인사관리 강화 지침」에서 “보안 상⋅벌\n',
 '점’ 인사관리 (진급심사 ) 반영”에 따라 서면경고 , 기타(불문, 구두경고 )에 \n',
 '대한 이의제기 절차와 보안벌점 ㆍ상점 처리절차를  지시하는  사항임 .\n',
 '2. 개정 배경\n',
 '   보안벌점 부과에 대한 이의제기 절차 및 연대급이하의 보안상점 건의 시 \n',
 '장관급 지휘관의 상점부여 내용을 추가하였음.\n',
 '3. 기본지시\n',
 '   가. 서면경고 , 기타(불문, 구두경고 )에 따라 보안벌점을 부과하는 절차 \n',
 '및 보안상점 관리절차는 다음과 같다. \n',
 ' ① 상ㆍ벌점\n',
 '    부 여\n',
 '  (각급 부대장 )⇨ ② 상ㆍ벌점\n',
 '   개인 통보\n',
 ' (이의제기 접수)⇨ ③ 보안상 ㆍ벌점\n',
 '   심의위 개최\n',
 ' (이의제기 접수)⇨ ④ 심의결과\n',
 ' 보고 / 개인통보\n',
 ' (장성급 부대장 )⇨ ⑤ 명령발령\xa0\xa0/\n',
 '   기변처리⇨ ⑥ 상ㆍ벌점\n',
 '    입력\n',
 '보안위반\n',
 '확인서\n',
 '(보안감사관 작성 \n',
 '/ 개인이 확인)보안부서에서\n',
 '개인에게 공문 또는\n',
 '서면 통보\n',
 '(이의제기서 접수 :\n',
 '직접수발 / 우편)이의제기서 접수 건\n',
 '(연대급 이하 종합 \n',
 '장성급부대 주관)보안처분심사결정\n',
 '통지서\n',
 '(보안부서 주관)국방인사정보체계\n',
 '(인사 실무자)국방인사정보체계\n',
 '(인사 실무자)\n',
 '나. 가항의 보안상 ・벌점 이의제기는 보안위반확인서를 받은 날 또는 \n',
 '처분을  안 날(통보를 받은 날)로부터 10일 이내에 한다. \n',
 '   다. 보안상 ․벌점 심의위는 이의제기 안건 및 연대급이하의 보안상

In [94]:
## 번호에 따라 분할하기
raw_txt = [txt for txt in raw_txt if txt!='\n']

raw = []
for txt in tqdm(raw_txt):
    if re.match('((\s*)\d+\.(.+)\s*(.+)\s*\n)|(부\s*칙)|(작\s*성\s*관)', txt):
        split_text = re.split(r'(\d+\.\s*[가-힣])', txt)  ## 편/장/절 중간에 조항이 있으면 쪼개기
        if len(split_text) >= 2:
            split_idx = [i for i, s in enumerate(split_text) if re.search(r'(\d+\.)',s)]
            for idx in sorted(split_idx, reverse=True):
                split_text[idx+1] = split_text[idx] + split_text[idx+1]
                del split_text[idx]
            split_text = [s for s in split_text if s]
            split_text = [re.sub(r'(\d+)\.\s*(.+)',r'제\1조(\2)', s) for s in split_text]
            raw.extend(split_text)
        else:
            raw.append(txt)
    else:
        split_text = re.split(r'(\d+\.\s*[가-힣])', txt)  ## 문장 중간에 조항이 있으면 쪼개기
        if len(split_text) >= 2:
            split_idx = [i for i, s in enumerate(split_text) if re.search(r'(\d+\.\s*[가-힣])',s)]
            for idx in sorted(split_idx, reverse=True):
                split_text[idx+1] = split_text[idx] + split_text[idx+1]
                del split_text[idx]
            split_text = [s for s in split_text if s]
            # raw 가 0일 경우
            if len(raw)==0:
                raw.extend(split_text)
            else:
                raw[-1] = "".join([raw[-1],split_text[0]])
                split_text = [re.sub(r'(\d+)\.\s*(.+)',r'제\1조(\2)', s) for s in split_text]
                raw.extend(split_text[1:])
        elif len(raw)==0:
            raw.append(txt)
        else:
            raw[-1] = "".join([raw[-1],txt])


raw


100%|██████████| 101/101 [00:00<00:00, 92757.76it/s]

['제1조(개    요)\n  「육규 200 군사보안규정 」(이하 “육규”라고 한다) 제194조의2 1항에 따\n른 보안벌점 및「보안위반자 인사관리 강화 지침」에서 “보안 상⋅벌\n점’ 인사관리 (진급심사 ) 반영”에 따라 서면경고 , 기타(불문, 구두경고 )에 \n대한 이의제기 절차와 보안벌점 ㆍ상점 처리절차를  지시하는  사항임 .\n',
 '제2조(개정 배경)\n   보안벌점 부과에 대한 이의제기 절차 및 연대급이하의 보안상점 건의 시 \n장관급 지휘관의 상점부여 내용을 추가하였음.\n',
 '제3조(기본지시)\n   가. 서면경고 , 기타(불문, 구두경고 )에 따라 보안벌점을 부과하는 절차 \n및 보안상점 관리절차는 다음과 같다. \n ① 상ㆍ벌점\n    부 여\n  (각급 부대장 )⇨ ② 상ㆍ벌점\n   개인 통보\n (이의제기 접수)⇨ ③ 보안상 ㆍ벌점\n   심의위 개최\n (이의제기 접수)⇨ ④ 심의결과\n 보고 / 개인통보\n (장성급 부대장 )⇨ ⑤ 명령발령\xa0\xa0/\n   기변처리⇨ ⑥ 상ㆍ벌점\n    입력\n보안위반\n확인서\n(보안감사관 작성 \n/ 개인이 확인)보안부서에서\n개인에게 공문 또는\n서면 통보\n(이의제기서 접수 :\n직접수발 / 우편)이의제기서 접수 건\n(연대급 이하 종합 \n장성급부대 주관)보안처분심사결정\n통지서\n(보안부서 주관)국방인사정보체계\n(인사 실무자)국방인사정보체계\n(인사 실무자)\n나. 가항의 보안상 ・벌점 이의제기는 보안위반확인서를 받은 날 또는 \n처분을  안 날(통보를 받은 날)로부터 10일 이내에 한다. \n   다. 보안상 ․벌점 심의위는 이의제기 안건 및 연대급이하의 보안상점 건의에 한하여 실시한다.\n   라. 나항의 기간이 도과한 경우 보안과는 인사처리 실무자에게 통보\n하고, 인사실무자가 국방인사정보체계를 통해 명령발령 및 기변처리 \n해야 한다.\n',
 '제4조(세부업무처리)\n   가. 보안벌점 부여\n      1) 보안벌점은 징계권을 보유한 대대급 이상 지휘관이 부

In [95]:
# 불필요한 값들 삭제
for i in range(len(raw)):
    raw[i] = re.sub(r'제\s*(\d+)\s*(편|장|절|관)',r'제 \1\2 ',raw[i])        # 공백 (숫자) 절 공백 제외 (예. 제 3 편)
    raw[i] = re.sub(r'제(\d+)조의 (\d+)',r'제\1조의\2',raw[i])            # n조의 2와 같은 항목 통일성 유지
    raw[i] = re.sub(r'부(\s)*칙(\s)*',r'부칙',raw[i])                     # 부칙 사이 공백 제거
    raw[i] = re.sub(r'제(\d+)조\[(.*)\]',r'제\1조(\2)',raw[i])            # 대괄호를 소괄호로 변경
    raw[i] = re.sub('\s+',' ',raw[i])                                    # 공백이 여러개일 경우 한개만 둠
    raw[i] = re.sub('(\s)*\n',' ',raw[i])                                # \n은 띄어쓰기로 변환
    raw[i] = re.sub('(\s)*\)',')',raw[i])                                # \n은 띄어쓰기로 변환
    raw[i] = re.sub('·',' ',raw[i])                                      # ·와 같은 특수문자는 나중에 규칙을 혼란스럽게 만들 수 있어 제외
    raw[i] = re.sub('&#\d*;',' ',raw[i])                                 # 특수문자 제외
    raw[i] = re.sub(' \(삭제\) ','(삭제)',raw[i])                         # 삭제조항 공백 제거
    raw[i] = re.sub('-\s+(\d+)\s+-','',raw[i])                           # page numbering 삭제
    raw[i] = re.sub('^\d+-\d+$','',raw[i])                               # 페이지 넘버링만 있는 리스트 삭제
    raw[i] = re.sub(r'\s*작\s*성\s*관', r'작성관', raw[i])                # 작성관으로 변경
    raw[i] = re.sub(r'국\s*방\s*부\s*장\s*관', r'', raw[i])               # 이제 필요 없으니 삭제
    raw[i] = re.sub(r'조\((.+?)\((.+?)\)(.+?)\)',r'조(\1<\2>\3)',raw[i])  # 조 안에 중첩괄호 <>으로 바꾸기

raw = [txt for txt in raw if txt] ## 빈 문자열 제거

In [96]:
raw

['제1조(개 요) 「육규 200 군사보안규정 」<이하 “육규”라고 한다> 제194조의2 1항에 따 른 보안벌점 및「보안위반자 인사관리 강화 지침」에서 “보안 상⋅벌 점’ 인사관리 (진급심사) 반영”에 따라 서면경고 , 기타(불문, 구두경고)에 대한 이의제기 절차와 보안벌점 ㆍ상점 처리절차를 지시하는 사항임 . ',
 '제2조(개정 배경) 보안벌점 부과에 대한 이의제기 절차 및 연대급이하의 보안상점 건의 시 장관급 지휘관의 상점부여 내용을 추가하였음. ',
 '제3조(기본지시) 가. 서면경고 , 기타<불문, 구두경고>에 따라 보안벌점을 부과하는 절차 및 보안상점 관리절차는 다음과 같다. ① 상ㆍ벌점 부 여 (각급 부대장)⇨ ② 상ㆍ벌점 개인 통보 (이의제기 접수)⇨ ③ 보안상 ㆍ벌점 심의위 개최 (이의제기 접수)⇨ ④ 심의결과 보고 / 개인통보 (장성급 부대장)⇨ ⑤ 명령발령 / 기변처리⇨ ⑥ 상ㆍ벌점 입력 보안위반 확인서 (보안감사관 작성 / 개인이 확인)보안부서에서 개인에게 공문 또는 서면 통보 (이의제기서 접수 : 직접수발 / 우편)이의제기서 접수 건 (연대급 이하 종합 장성급부대 주관)보안처분심사결정 통지서 (보안부서 주관)국방인사정보체계 (인사 실무자)국방인사정보체계 (인사 실무자) 나. 가항의 보안상 ・벌점 이의제기는 보안위반확인서를 받은 날 또는 처분을 안 날(통보를 받은 날)로부터 10일 이내에 한다. 다. 보안상 ․벌점 심의위는 이의제기 안건 및 연대급이하의 보안상점 건의에 한하여 실시한다. 라. 나항의 기간이 도과한 경우 보안과는 인사처리 실무자에게 통보 하고, 인사실무자가 국방인사정보체계를 통해 명령발령 및 기변처리 해야 한다. ',
 '제4조(세부업무처리) 가. 보안벌점 부여 1) 보안벌점은 징계권을 보유한 대대급 이상 지휘관이 부여하고 , 관련 심의는 차상급부대인 연대 및 장관급부대에서 진행한다. 2) 서면경고 , 기타<불문, 구두경고>시 벌점을 부과하는 경우에는 대대급 이상 각급부대장 (보안담당관)이 보안위반확인서를 양식(별지 #1)에

In [97]:
# pattern = '(제 [0-9]{1,3} 편)+(\s*\w*)*(제 [0-9]{1,3} 장)*(\s*\w*)*(제[0-9]{1,3}절)*(\s*\w*)*(제[0-9]{1,3}조)+\((\s*\w*)+\)'

pattern = r'제[0-9]{1,3}조(의\s*\d+\s*)?\s*\((.+?)\)|(부칙)|(작성관)'
name_com = re.compile(pattern)

# candidate pool jsonl 생성
candidate_pool = []
for i, txt in tqdm(enumerate(raw)):
    try:
        split_std = name_com.search(txt).span()[1]  # 이름과 제목 끊을 라인
        txt_dic = {
            'subject':txt[:split_std],
            'text':txt[split_std:],
            'reference': txt_file_name.replace('.txt','')
            }
        candidate_pool.append(txt_dic)
    except AttributeError:
        print(str(i)+'행의"'+ txt + '" 문장을 확인해주세요.')
        pass

8it [00:00, 28102.54it/s]


In [99]:
candidate_pool

[{'subject': '제1조(개 요)',
  'text': ' 「육규 200 군사보안규정 」<이하 “육규”라고 한다> 제194조의2 1항에 따 른 보안벌점 및「보안위반자 인사관리 강화 지침」에서 “보안 상⋅벌 점’ 인사관리 (진급심사) 반영”에 따라 서면경고 , 기타(불문, 구두경고)에 대한 이의제기 절차와 보안벌점 ㆍ상점 처리절차를 지시하는 사항임 . ',
  'reference': '17-3001'},
 {'subject': '제2조(개정 배경)',
  'text': ' 보안벌점 부과에 대한 이의제기 절차 및 연대급이하의 보안상점 건의 시 장관급 지휘관의 상점부여 내용을 추가하였음. ',
  'reference': '17-3001'},
 {'subject': '제3조(기본지시)',
  'text': ' 가. 서면경고 , 기타<불문, 구두경고>에 따라 보안벌점을 부과하는 절차 및 보안상점 관리절차는 다음과 같다. ① 상ㆍ벌점 부 여 (각급 부대장)⇨ ② 상ㆍ벌점 개인 통보 (이의제기 접수)⇨ ③ 보안상 ㆍ벌점 심의위 개최 (이의제기 접수)⇨ ④ 심의결과 보고 / 개인통보 (장성급 부대장)⇨ ⑤ 명령발령 / 기변처리⇨ ⑥ 상ㆍ벌점 입력 보안위반 확인서 (보안감사관 작성 / 개인이 확인)보안부서에서 개인에게 공문 또는 서면 통보 (이의제기서 접수 : 직접수발 / 우편)이의제기서 접수 건 (연대급 이하 종합 장성급부대 주관)보안처분심사결정 통지서 (보안부서 주관)국방인사정보체계 (인사 실무자)국방인사정보체계 (인사 실무자) 나. 가항의 보안상 ・벌점 이의제기는 보안위반확인서를 받은 날 또는 처분을 안 날(통보를 받은 날)로부터 10일 이내에 한다. 다. 보안상 ․벌점 심의위는 이의제기 안건 및 연대급이하의 보안상점 건의에 한하여 실시한다. 라. 나항의 기간이 도과한 경우 보안과는 인사처리 실무자에게 통보 하고, 인사실무자가 국방인사정보체계를 통해 명령발령 및 기변처리 해야 한다. ',
  'reference': '17-3001'},
 {'subject': '제

In [34]:
# candidate pool jsonl 형태로 저장
with open('./candidate_pool/'+ txt_file_name.replace('.txt','.jsonl'),encoding='utf-8', mode="w") as file:
    for i in candidate_pool:
        file.write(json.dumps(i, ensure_ascii=False, indent = 3) + "\n")

In [ ]:
# candidate pool csv 형태로 저장
import pandas as pd
pd.DataFrame(candidate_pool).to_csv('./candidate_pool/'+file_name+'.csv', encoding='euc-kr')

In [ ]:
import jsonlines

# with jsonlines.open('./candidate_pool/'+ file_name + '.jsonl') as f:
#     for line in f.iter():
#         print(line["id"]) # 각 json에 해당하는 "id" 출력
        # print(line["title"]) # 각 json에 해당하는 "title" 출력

In [ ]:
f = jsonlines.open('./candidate_pool/'+ file_name + '.jsonl')

In [ ]:
# 확인 
# with open('./candidate_pool/candidate_pool'+ file_name + ".jsonl") as f: 
# 	for line in f: print(line)